In [ ]:
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import json
from itertools import cycle

from rasterio.crs import CRS
from rasterio.plot import show
from rasterio.warp import transform_geom
from rasterio.features import shapes
import rasterio
from shapely import from_geojson
from shapely.geometry import mapping, shape, MultiPolygon
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from geopandas import GeoDataFrame
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import folium

from src.notebook_helper import *

# Set dirctory paths

In [ ]:
BASE = Path("/Users/kyledorman/data/plant_download")
GRIDS_DIR = BASE / "wgs84_all_geojson"
GRID_PATHS = [p for p in GRIDS_DIR.iterdir() if p.suffix == ".geojson"]

# Inspect All Grids

In [ ]:
# Load grids
grids = [gpd.read_file(p) for p in GRID_PATHS]

# Combine into a single GeoDataFrame
global_gdf = gpd.GeoDataFrame(pd.concat(grids, ignore_index=True))

# Convert to local CRS for centroid calculations
local_gdf = global_gdf.to_crs(global_gdf.estimate_utm_crs())

In [ ]:
# Step 1: Sort by centroid position
global_gdf['centroid_x'] = local_gdf.geometry.centroid.x
global_gdf['centroid_y'] = local_gdf.geometry.centroid.y
global_gdf = global_gdf.sort_values(
    by=['centroid_y', 'centroid_x'], 
    ascending=[True, True]
).reset_index(drop=True)

# Step 2: Assign colors cyclically
# Generate a large color palette
colors = plt.get_cmap('tab20', 20)  # 20 distinct colors (use larger if needed)
color_palette = [colors(i) for i in range(colors.N)]
hex_colors = [matplotlib.colors.rgb2hex(c[:3]) for c in color_palette]  # Convert to HEX
color_cycle = cycle(hex_colors)

# Assign colors
global_gdf['color'] = [next(color_cycle) for _ in range(len(global_gdf))]

In [ ]:
# Compute the bounding box of all polygons
minx, miny, maxx, maxy = global_gdf.total_bounds

# Calculate the center of the bounding box
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

# Calculate dynamic zoom level
zoom_level = calculate_zoom_level(global_gdf.total_bounds)

# Create the base map centered on the calculated location
base_map = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_level, width=800, height=600)

# Add each GeoJSON file to the map
# Add polygons to the map
for _, row in global_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=row['color']: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5,
        },
    ).add_to(base_map)
    
# Display the map
base_map

# Inspect UDM extents compared to an AOI

Set the GRID_ID, YEAR, MONTH, and RESULTS_BASE variables

In [ ]:
GRID_ID = "25058022"
YEAR = "2022"
MONTH = "09"
RESULTS_BASE = BASE / "test3" / "results" / YEAR / MONTH

In [ ]:
GRID = gpd.read_file(GRIDS_DIR / f"{GRID_ID}.geojson")
RESULTS_GRID_DIR = RESULTS_BASE / GRID_ID
UDM_PATHS = list((RESULTS_GRID_DIR / "udm").iterdir())

In [ ]:
# Load the GeoJSON file
geojson_file = RESULTS_GRID_DIR / "search_geometries.json"
gdf = gpd.read_file(geojson_file)

zoom_level = calculate_zoom_level(GRID.total_bounds)
m = folium.Map(location=(GRID.centroid.iloc[0].y, GRID.centroid.iloc[0].x), zoom_start=zoom_level - 2)

# Add AOI to the map in blue
folium.GeoJson(
    GRID,
    name="AOI",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "blue",
        "weight": 2,
        "fillOpacity": 0.9,
    },
).add_to(m)

# Plot each polygon with a different color
for _, row in gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=next(color_cycle): {
            'fillColor': color,
            'color': color,
            'weight': 2,
            'fillOpacity': 0.05,
        }
    ).add_to(m)



# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

# Inspect downloaded grid images

To inspect the downloaded imagery, first run
```bash
python src/inspect_grid_outputs.py --month MONTH --year YEAR --config-file CONFIG_FILE --grid-id GRID-ID
```

In [ ]:
asset_dir = RESULTS_BASE / GRID_ID / 'files_asset_cropped'
udm_dir = RESULTS_BASE / GRID_ID / 'files_udm_cropped'
image_paths = sorted(list(asset_dir.iterdir()))
udm_paths = sorted(list(udm_dir.iterdir()))

cols = 2
rows = len(image_paths) // cols + len(image_paths) % cols
image_size = 5

fig, axes = plt.subplots(rows, cols, figsize=(image_size * cols, image_size * rows))
for ax in axes.flatten():
    ax.axis('off')

for i, (img_pth, udm_path) in enumerate(zip(image_paths, udm_paths)):
    row = i // cols
    col = i % cols
    with rasterio.open(img_pth) as src:
        img = src.read((7, 5, 3), masked=True)
        
    with rasterio.open(udm_path) as src:
        udm = (src.read(1) == 1).astype(np.uint8)
        
    # img = np.where(udm != 1, 0, img)
    v_min, v_max = np.percentile(img.compressed(), (2, 98))
    img = np.clip((img - v_min) / (v_max - v_min), 0, 1)
    show(img, ax=axes[row, col])

plt.tight_layout()

In [ ]:
# Create counter
with rasterio.open(udm_paths[0]) as src:
    img = (src.read(1) == 1).astype(np.uint8)
    counter = np.zeros_like(img)

for file in udm_paths:
    with rasterio.open(file) as src:
        img = (src.read(1) == 1).astype(np.uint8)
        counter += img

show(counter, cmap='inferno',  title="Counts")
_ = show(counter.clip(0, 5), cmap='inferno', title="Clipped Counts")